# DR missing files
- check sessions one-by-one
- mark each session as checked when finished


Problem sessions:
- `_20220927` (timestamps file needs recovering)
- `_20220825` (half-recording)

In [ ]:
import time
import pprint
from typing import Generator

from np_session import Session, sessions as get_sessions

while not (USERNAME := input('Enter username:')):
    continue

problem_sessions = ('_20220825', '_20220927')

def sessions(skip_already_checked=True) -> Generator[Session, None, None]:
    for session in get_sessions():
        if any(_ in str(session) for _ in problem_sessions):
            continue
        if skip_already_checked and session.state.get('checked'):
            continue
        yield session

DR_sessions: Generator[Session, None, None] = sessions()

# Get next DR session:

In [ ]:
session = next(DR_sessions)

# Display files missing on np-exp:
can be re-run if files have changed 

In [ ]:
pprint.pprint(missing := session.get_missing_files())

---
---
# Mark session as checked when all files are present

Acceptable missing files:
- raw data already in lims:
    - `*_probeABC`
    - `*_probeDEF` 
- large sorted files already deleted:
    - `*_sorted/ ... /continuous.dat`
- surface images 

In [ ]:
session.state['checked'] = time.time()
session.state['checked_by'] = USERNAME 